In [5]:
!pip install -r requirements.txt

ERROR: Ignored the following versions that require a different python version: 1.21.2 Requires-Python >=3.7,<3.11; 1.21.3 Requires-Python >=3.7,<3.11; 1.21.4 Requires-Python >=3.7,<3.11; 1.21.5 Requires-Python >=3.7,<3.11; 1.21.6 Requires-Python >=3.7,<3.11; 1.26.0 Requires-Python <3.13,>=3.9; 1.26.1 Requires-Python <3.13,>=3.9
ERROR: Could not find a version that satisfies the requirement mediapipe (from versions: none)

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for mediapipe


# Starter code

### Clean directory

In [1]:
import shutil
from pathlib import Path

def clean_directory(dir_path: Path):
    """Remove all files and subdirectories inside the given directory."""
    if not dir_path.exists():
        print(f"⚠️ Directory {dir_path} does not exist.")
        return

    if not dir_path.is_dir():
        raise ValueError(f"{dir_path} is not a directory")

    for item in dir_path.iterdir():
        if item.is_file() or item.is_symlink():
            item.unlink()  # delete file or symlink
        elif item.is_dir():
            shutil.rmtree(item)  # delete folder recursively

    print(f"✅ Cleaned directory: {dir_path}")

### Creating a test sample

In [2]:
import random
import shutil
from pathlib import Path

def create_test_sample(image_dir: Path, output_dir: Path, sample_size: int):
    # --- Collect all images ---
    images = sorted(
        [p for p in image_dir.iterdir() if p.suffix.lower() in {".jpg", ".png"}]
    )

    if not images:
        raise ValueError(f"No images found in {image_dir}")

    # --- Ensure sample size is valid ---
    if sample_size > len(images):
        raise ValueError(f"Sample size {sample_size} is larger than available images ({len(images)})")

    # --- Pick random sample ---
    sampled_images = random.sample(images, sample_size)

    # --- Create output folder ---
    output_dir.mkdir(parents=True, exist_ok=True)
    clean_directory(output_dir)

    # --- Copy files ---
    for img in sampled_images:
        shutil.copy(img, output_dir / img.name)

    print(f"✅ Copied {sample_size} images to {output_dir}")

# -------------------------------
# Example usage
# -------------------------------

if __name__ == "__main__":
    image_dir = Path(r"C:\PoiseVideos\4825_Prasanna K.B_189_20250915225127\4825_Prasanna K.B_189_20250915225127")                  # folder with images
    output_dir = Path("test_sample")    # new folder to be created
    sample_size = 100                  # number of images to copy

    create_test_sample(image_dir, output_dir, sample_size)


✅ Cleaned directory: test_sample
✅ Copied 100 images to test_sample


---

In [3]:
import cv2
import mediapipe as mp
import numpy as np
from pathlib import Path

# === CONFIG (Global variables you can tweak) ===
INPUT_DIR = Path("test_sample")    # Folder containing jpgs
OUTPUT_DIR = Path(r"output\convex_hull_output\results")  # Folder for background-removed images
DEBUG_DIR = Path(r"output\convex_hull_output\debug")    # Folder for images with landmarks + hull
PADDING = 40                        # Padding around convex hull (in pixels)

# Create output folders if they don’t exist
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
DEBUG_DIR.mkdir(parents=True, exist_ok=True)

# Mediapipe Pose setup
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(static_image_mode=True, model_complexity=2)


def get_body_mask(image, landmarks, padding=PADDING):
    h, w, _ = image.shape

    # Collect landmark points
    points = []
    for lm in landmarks.landmark:
        x, y = int(lm.x * w), int(lm.y * h)
        points.append([x, y])
    points = np.array(points)

    # Convex hull
    hull = cv2.convexHull(points)

    # Add padding by dilating hull mask
    mask = np.zeros((h, w), dtype=np.uint8)
    cv2.fillConvexPoly(mask, hull, 255)
    kernel = np.ones((padding, padding), np.uint8)
    mask = cv2.dilate(mask, kernel, iterations=1)

    return mask, hull, points


def process_image(img_path, out_path, debug_path):
    image = cv2.imread(str(img_path))
    if image is None:
        print(f"⚠️ Could not read {img_path}")
        return

    # Run pose detection
    results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    if not results.pose_landmarks:
        print(f"⚠️ No person detected in {img_path}")
        # Save a black background image
        black_bg = np.zeros_like(image)
        cv2.imwrite(str(out_path), black_bg)
        cv2.imwrite(str(debug_path), black_bg)
        return

    # Get mask + convex hull + points
    mask, hull, points = get_body_mask(image, results.pose_landmarks)

    # === Background removed result ===
    result = np.zeros_like(image)
    result[mask == 255] = image[mask == 255]
    cv2.imwrite(str(out_path), result)

    # === Debug image with landmarks + hull ===
    debug_img = image.copy()

    # Draw landmarks with connections
    mp_drawing.draw_landmarks(
        debug_img,
        results.pose_landmarks,
        mp_pose.POSE_CONNECTIONS,
        mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
        mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2),
    )

    # Draw convex hull
    cv2.polylines(debug_img, [hull], isClosed=True, color=(255, 0, 0), thickness=2)

    cv2.imwrite(str(debug_path), debug_img)


def main():
    for img_path in INPUT_DIR.glob("*.jpg"):
        out_path = OUTPUT_DIR / img_path.name
        debug_path = DEBUG_DIR / img_path.name
        process_image(img_path, out_path, debug_path)
        print(f"✅ Processed: {img_path.name}")


if __name__ == "__main__":
    main()


ModuleNotFoundError: No module named 'cv2'

In [4]:
!pip list

Package                   Version
------------------------- --------------
anyio                     4.10.0
argon2-cffi               25.1.0
argon2-cffi-bindings      25.1.0
arrow                     1.3.0
asttokens                 3.0.0
async-lru                 2.0.5
attrs                     25.3.0
babel                     2.17.0
beautifulsoup4            4.13.5
bleach                    6.2.0
certifi                   2025.8.3
cffi                      2.0.0
charset-normalizer        3.4.3
checksumdir               1.2.0
colorama                  0.4.6
comm                      0.2.3
debugpy                   1.8.16
decorator                 5.2.1
defusedxml                0.7.1
executing                 2.2.1
fastjsonschema            2.21.2
fqdn                      1.5.1
git-filter-repo           2.47.0
h11                       0.16.0
httpcore                  1.0.9
httpx                     0.28.1
idna                      3.10
ipykernel                 6.30.1
ipython        